### Single and Multi Headed attention

- Query, key, value vectors are not identical in practice.
- They are each a linear transformation of embedding vector.
- This allows self attention layer to focus on complementary words in the context rather than the same word itself.
- Linear transformation is done with learnable parameters.
- In single headed attn, only transformation is created for each of query, key and value
- In multi headed attn, multiple transformations are created.
- Each head encode a different aspect of similarity like subject-verb interaction, nearby adjectives etc.
- These as aspects are fully learned from data and are similar to filters in CNNs.


Single headed attention


In [3]:
from transformers import AutoTokenizer, AutoConfig

from torch import nn, bmm
import numpy as np
from math import sqrt

In [4]:
def scaled_dot_prod_attn(query, key, value):

    dim_k = query.size(-1)
    scores = bmm(query, key.transpose(1,2))/sqrt(dim_k)
    weights = nn.functional.softmax(scores, dim=-1)
    attn_outputs = bmm(weights, value)
    return attn_outputs

One attention head

embed_dim:

head_dim:

In [5]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim) -> None:
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, hidden_state):
        attn_outputs = scaled_dot_product_attention(self.q(hidden_state),
                                                    self.k(hidden_state),
                                                    self.v(hidden_state))
        return attn_outputs

Multi attention heads

In [ ]:
class MultiAttentionHead(nn.module):
    def __init__(self, config) -> None:
        super().__init__()
        
